In [1]:
from libsvm.svmutil import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [2]:
x_train = np.load('./x_train.npy',allow_pickle=True)
y_train = np.load('./y_train.npy',allow_pickle=True)
x_test_PIE = np.load('./x_test_PIE.npy',allow_pickle=True)
y_test_PIE = np.load('./y_test_PIE.npy',allow_pickle=True)
x_test_slf = np.load('./x_test_slf.npy',allow_pickle=True)
y_test_slf = np.load('./y_test_slf.npy',
                     allow_pickle=True).reshape(-1)
x_test = np.vstack((x_test_PIE, x_test_slf))
y_test = np.hstack((y_test_PIE, y_test_slf))

In [3]:
# 将数据保存为 LibSVM 格式
def save_libsvm_data(x, y, file_path):
    with open(file_path, 'w') as file:
        for i in range(len(x)):
            label = str(y[i])
            features = ' '.join([f"{index + 1}:{value}" 
                                 for index, value in enumerate(x[i])])
            line = f"{label} {features}\n"
            file.write(line)

# 保存数据到文件
save_libsvm_data(x_train, y_train, './svm/libsvm_train.txt')
save_libsvm_data(x_test, y_test, './svm/libsvm_test.txt')

In [4]:
y_train, X_train = svm_read_problem('./svm/libsvm_train.txt')
C = ['1e-2', '1e-1', '1']
for c in C:
    model = svm_train(y_train, X_train, '-c '+ c +' -t 0')
    y_test, X_test = svm_read_problem('./svm/libsvm_test.txt')
    print('c = ',c, 'input to SVM is raw')
    p_labels, p_acc, p_vals = svm_predict(y_test, X_test, model)
    print('-------------------------------------------------')

c =  1e-2 input to SVM is raw
Accuracy = 85.759% (1096/1278) (classification)
-------------------------------------------------
c =  1e-1 input to SVM is raw
Accuracy = 85.759% (1096/1278) (classification)
-------------------------------------------------
c =  1 input to SVM is raw
Accuracy = 85.759% (1096/1278) (classification)
-------------------------------------------------


In [10]:
def PCA_decomponent(img):
    img -= np.mean(img, axis=0)
    cov = np.dot(img.T, img)
    values,vecs = np.linalg.eig(cov)
    V = vecs
    vec = vecs[0:200]
    return vec

def PCA_projection(img, vec):
    img -= np.mean(img, axis=0)
    return img @ vec.T

vec = PCA_decomponent(x_train)
X_train_projection = PCA_projection(img=x_train, vec=vec).real
x_test_projection = PCA_projection(img=x_test, vec=vec).real
D = [80,200]
for d in D:
    print('input to SVM is after PCA, dim =',d)
    for c in C:
        model = svm_train(y_train, X_train_projection[:,:d], '-c '+ c +' -t 0')
        y_test, X_test = svm_read_problem('./svm/libsvm_test.txt')
        print('     c = ',c,':')
        p_labels, p_acc, p_vals = svm_predict(y_test, x_test_projection, model)
    print('-------------------------------------------------')


input to SVM is after PCA, dim = 80
     c =  1e-2 :
Accuracy = 80.6729% (1031/1278) (classification)
     c =  1e-1 :
Accuracy = 80.6729% (1031/1278) (classification)
     c =  1 :
Accuracy = 80.6729% (1031/1278) (classification)
-------------------------------------------------
input to SVM is after PCA, dim = 200
     c =  1e-2 :
Accuracy = 84.9765% (1086/1278) (classification)
     c =  1e-1 :
Accuracy = 84.9765% (1086/1278) (classification)
     c =  1 :
Accuracy = 84.9765% (1086/1278) (classification)
-------------------------------------------------
